In [8]:
#imports
import pandas as pd
import numpy as np

In [10]:
%pip install openpyxl

# 1. Load both datasets
transactions = pd.read_excel("data/customer_transactions.xlsx")
social_profiles = pd.read_excel("data/customer_social_profiles.xlsx")
# 2. Standardize column names
transactions.columns = [
    "customer_id_legacy", "transaction_id", "purchase_amount",
    "purchase_date", "product_category", "customer_rating"
]

social_profiles.columns = [
    "customer_id_new", "social_media_platform",
    "engagement_score", "purchase_interest_score", "review_sentiment"
]

# 3. Clean and align customer IDs


# Remove any non-numeric characters and convert to integers
transactions["customer_id_clean"] = transactions["customer_id_legacy"].astype(str).str.extract(r"(\d+)").astype(int)
social_profiles["customer_id_clean"] = social_profiles["customer_id_new"].astype(str).str.extract(r"(\d+)").astype(int)


# 4. Handle missing values


# Check for missing values
print("Missing values before cleaning:")
print("Transactions:\n", transactions.isnull().sum())
print("\nSocial Profiles:\n", social_profiles.isnull().sum())

# Fill or drop missing values where appropriate
transactions["customer_rating"] = pd.to_numeric(transactions["customer_rating"], errors="coerce")
transactions["customer_rating"].fillna(transactions["customer_rating"].mean(), inplace=True)

social_profiles["engagement_score"] = pd.to_numeric(social_profiles["engagement_score"], errors="coerce")
social_profiles["purchase_interest_score"] = pd.to_numeric(social_profiles["purchase_interest_score"], errors="coerce")

social_profiles["engagement_score"].fillna(social_profiles["engagement_score"].mean(), inplace=True)
social_profiles["purchase_interest_score"].fillna(social_profiles["purchase_interest_score"].mean(), inplace=True)
social_profiles["review_sentiment"].fillna("Neutral", inplace=True)


# 5. Remove duplicates

transactions.drop_duplicates(subset=["transaction_id"], inplace=True)
social_profiles.drop_duplicates(subset=["customer_id_clean", "social_media_platform"], inplace=True)


# 6. Merge datasets

merged_df = pd.merge(
    transactions,
    social_profiles,
    on="customer_id_clean",
    how="inner"
)


# 7. Final cleanup


# Drop redundant columns
merged_df.drop(columns=["customer_id_legacy", "customer_id_new"], inplace=True)

# Convert date column to datetime
merged_df["purchase_date"] = pd.to_datetime(merged_df["purchase_date"], errors="coerce")

# Reorder columns logically
merged_df = merged_df[
    [
        "customer_id_clean", "transaction_id", "purchase_date", "product_category",
        "purchase_amount", "customer_rating", "social_media_platform",
        "engagement_score", "purchase_interest_score", "review_sentiment"
    ]
]


# 8. Save cleaned dataset
output_path = "data/cleaned_customer_data.csv"
merged_df.to_csv(output_path, index=False)

print("Cleaning and merge complete.")
print(f"Rows: {merged_df.shape[0]}, Columns: {merged_df.shape[1]}")
print(f"Saved cleaned dataset to: {output_path}")
display(merged_df.head())


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]

Note: you may need to restart the kernel to use updated packages.
Missing values before cleaning:
Transactions:
 customer_id_legacy     0
transaction_id         0
purchase_amount        0
purchase_date          0
product_category       0
customer_rating       10
customer_id_clean      0
dtype: int64

Social Profiles:
 customer_id_new            0
social_media_platform      0
engagement_score           0
purchase_interest_score    0
review_sentiment           0
customer_id_clean          0
dtype: int64
Cleaning and merge complete.
Rows: 187, Columns: 10
Saved cleaned dataset to: data/cleaned_customer_data.csv


C:\Users\Hp\AppData\Local\Temp\ipykernel_7188\3142154023.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  transactions["customer_rating"].fillna(transactions["customer_rating"].mean(), inplace=True)
C:\Users\Hp\AppData\Local\Temp\ipykernel_7188\3142154023.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

,customer_id_clean,transaction_id,purchase_date,product_category,purchase_amount,customer_rating,social_media_platform,engagement_score,purchase_interest_score,review_sentiment
0,151,1001,2024-01-01,Sports,408,2.3,TikTok,61,1.3,Neutral
1,151,1001,2024-01-01,Sports,408,2.3,Twitter,72,1.6,Neutral
2,192,1002,2024-01-02,Electronics,332,4.2,Instagram,60,4.3,Positive
3,114,1003,2024-01-03,Electronics,442,2.1,Facebook,87,4.8,Negative
4,160,1005,2024-01-05,Clothing,64,1.3,TikTok,56,2.6,Positive
